In [2]:
import mysql.connector
import json

with open("secrets.json", 'r') as f:
    data = json.load(f)
    connection = mysql.connector.connect(
        host=data['host'],
        database=data["database"],
        user=data['cpanel_user'],
        port="3306",
        passwd=data['cpanel_pass']
    )
    del data # We want to not keep .secrets in memory :) 
cursor = connection.cursor()

Opening Connection.


AttributeError: 'SqlInteractions' object has no attribute '_SqlInteractions__curr'

In [4]:
from time import sleep # Need to make sure we pause for a second after every API call
import requests
import json
import logging
import re
# Setup error Logging.
logging.basicConfig(filename='requestsError.log', encoding='utf-8', level=logging.ERROR)





class GameSharkAPI:
    def __init__(self):
        self.baseUrl = "https://www.cheapshark.com/api/1.0/"
        self.baseImgUrl = "https://www.cheapshark.com/"
        self.payload={}
        self.headers = {}
    
    def requestTest(self, url="https://www.cheapshark.com/api/1.0/deals?storeID=1&upperPrice=15"):
        """Function to test and list the data from the testing url
        Will print the first 5 of the request only, otherwise will log an error
        
        Keyword arguments:
        url: Url to test, base one is the test one from the website.
        """
        jsonData = self.requestUrl(url).json()
        if jsonData != None: 
            self.printJson(jsonData[:5])
        return jsonData

    def requestUrl(self, url)  :
        """Class to pass in a url to get the json data back.

        Keyword arguments:
        url: Url for the location we'd like to request data from.
        
        Returns:
        On Success -> json(dictionary) of the requested data
        On Failure -> None
        """
        try:
            sleep(1) # Pauses for 1 second in between any call
            response = requests.request("GET", url, headers=self.headers, data=self.payload)
            assert 200 == response.status_code
            return response
        except:
            logging.error(f"ERROR: Did not recieve code 200, recieved code {response.status_code}; URL: {url}")
            print(f"ERROR: Did not recieve code 200, recieved code {response.status_code}; URL: {url}")
            return None# Return None if we have an error.
    
    def printJson(self, jsonData: dict):
        """Prints the contents of a request.
        
        Keyword arguments:
        jsonData: a dictionary that contains the contents of a file to print. 
        """
        for data in jsonData:
            print(json.dumps(data, indent=4))

    def goToDeal(self, dealId:str):
        """Takes in a dealId and sends you to the link through
        cheapsharks portal (Used to help them get a credit for the
        sale, if bought, terms of using the API for free.)
        """

        dealId="liaqeBQtozzs0vzBu8CY9pN78c4rUdHcSdL2SnDowhA%3D"
        dealUrl = f"https://www.cheapshark.com/redirect?dealID={dealId}"
        
        # This will be replaced later with functionality in streamlit
        # NOTE: This is really slow to open a browser in WSL.
        #       Keep the WSL chrome browser open after first run.
        #       For faster testing
        
        import webbrowser  
        webbrowser.open(dealUrl, new=0, autoraise=True)
    
    def getAllDeals(self):
        """Get all deals from Cheapshark Api.
        TODO: 
            1. This will feed the data into PySpark. 
            2. Figure out how to get the entire list, since you get the data in pages.
        """
        initRun = self.baseUrl + f"deals?"
    
    def getStoreIds(self):
        storeUrl = self.baseUrl + "stores"
        jsonObj = self.requestUrl(storeUrl)
        #self.printJson(jsonData=jsonObj.json())

        # Cleanse the Json Data to be inserted into MySQL
        listOfJson = []
        if jsonObj:
            for data in jsonObj.json():
                data['storeID'] = int(data['storeID'])
                newData = data | {key : self.baseImgUrl + row[1:] for key, row in data["images"].items()}
                del newData['images']
                listOfJson.append(newData)
            return listOfJson
        return listOfJson
                
        
    def getMaxDealPages(self, **argv):
        """
        Uses the test to get the max amount of pages.
        1. If title is given, get for that title.
        ...

        2. Cleanse the returned output for insert into list.

        TODO: 
            Use
            data.headers['X-Total-Page-Count']
            To loop through multiple pages.

        
        """
        
        # If a title was given, add.
        title = "" if not argv.get("title") else f"&title={argv['title']}"
        pageNum = 0
        jsonObj = self.requestUrl(url=f"https://www.cheapshark.com/api/1.0/deals?pageSize=60&pageNumber={pageNum}{title}")
        if jsonObj == None:
            return [] # return an empty list if its an empty query.
        
        # TODO: Add here a for loop to keep iterating over pages.

        convertDict = {
            # Old Key        # New Key # Cast type
            "internalName" : ("gameKey", str),
            "storeID" : ("storeID", int),
            "title"   : ("title", str),
            "gameID"  : ("gameID", str),
            "salePrice" : ("salePrice", float),
            "normalPrice" : ("salePrice", float),
            "isOnSale" : ("isOnSale", int),
            "metacriticScore" : ("metacriticScore", int),
            "steamRatingText" : ("steamRatingText", str),
            "steamRatingPercent" : ("steamRatingPercent", int),
            "steamRatingCount" : ("steamRatingCount", int),
            "releaseDate" : ("releaseDate", str),
            "lastChange" : ("lastChange", str),
            "thumb" : ("thumb", str)
        }
        
        listOfJson = []
        subList = []
        count = 0
        for data in jsonObj.json():
            newObj = {}
            # Iterate over the data.
            for oldTitle, (newKey, castType) in convertDict.items():
                newObj[newKey] = castType(data[oldTitle]) 
            subList.append(newObj)
            count += 1
            # Split into lists of max 800, so that we dont overload the database.
            if count >= 800:
                listOfJson.append(subList.copy())
                subList = []
                count = 0
        if subList:
            listOfJson.append(subList)
        return listOfJson
            

    def searchGame(self, userInput):
        """Runs a query to the API to search for a game given a parameter.
        converts the userInput into a format the API recognizes.
        """
        cleansedInput = userInput.strip().replace(" ", "%20").upper()
        print(cleansedInput)
        return (self.getMaxDealPages(title=userInput), cleansedInput)
        


class SqlInteractions:
    
    def __init__(self):
        """
        Launches the sql object
        Initializes core functionality and creates a self.__cur object.
        """
        self.gsa = GameSharkAPI()

    def nonCommitWrapper(func):
        """
        Singular Input, resets the 
        """
        def wrapper(self, *x):
            with open(".secrets.json", 'r') as f:
                data = json.load(f)
                connection = mysql.connector.connect(
                    host=data['host'],
                    database=data["database"],
                    user=data['cpanel_user'],
                    port=data["port"],
                    passwd=data['cpanel_pass']
                )
                del data # We want to not keep .secrets in memory :) 
            self.__cur = connection.cursor()
            
            print("Opening Connection.")
            if len(x) > 0:
                func(self, *x)
            else:
                func(self)
            print("Closing Connection.")
            self.__cur.close()
        return wrapper



    def commitWrapper(func):
        """
        Singular Input
        PLEASE TEST THIS BEFORE USE!!!
        THIS WAS CREATED HAPHAZARDLY
        """
        def wrapper(self, *x):
            with open(".secrets.json", 'r') as f:
                data = json.load(f)
                self.connection = mysql.connector.connect(
                    host=data['host'],
                    database=data["database"],
                    user=data['cpanel_user'],
                    port=data["port"],
                    passwd=data['cpanel_pass']
                )
                del data # We want to not keep .secrets in memory :) 
            self.connection.autocommit = True
            self.__cur = self.connection.cursor()
            
            print("Opening Connection.")
            if len(x) > 0:
                func(self, *x)
            else:
                func(self)
            self.connection.commit()
            print("Commited\nClosing Connection.")
            self.__cur.close()
            self.connection.close()
        return wrapper


    @nonCommitWrapper
    def getTableNames(self):
        """
        Used to return the table names for the class to function properly
        """
        self.__curr.execute("""
        SHOW TABLES;
        """)
        self.__curr.fetchall()
    
    @nonCommitWrapper
    def getTableNames2(self, z):
        """
        Used to return the table names for the class to function properly
        """
        
    def getTableCopy(self, tableName):
        pass

    @commitWrapper
    def updateStoreDB(self):
        """
        Takes the read in data from the cheapshark API and stores the data in MySQL through 
        insertStoresJsonIn
        """
        dataSet = self.gsa.getStoreIds()
        jsonData = json.dumps(dataSet)
        print(jsonData) 
        self.__cur.callproc('insertStoresJsonIn', [jsonData])
        
    @nonCommitWrapper
    def updateSearchDB(self, searchInput):
        """
        Takes in a users search, and will call the stored procedure
        to update both the userSearches table and the General Product Database
        table.
        """

        pass


# test = SqlInteractions()
# test.updateStoreDB()
test2 = GameSharkAPI()
test2.searchGame("Dragon Quest")

DRAGON%20QUEST
{
    "internalName": "DRAGONQUESTBUILDERS2",
    "title": "DRAGON QUEST BUILDERS 2",
    "metacriticLink": "/game/pc/dragon-quest-builders-2",
    "dealID": "b7vOKGrh7NZP%2BW0UmVE%2BXrpRGOSrksLcJFNyMriyXgY%3D",
    "storeID": "1",
    "gameID": "207284",
    "salePrice": "24.99",
    "normalPrice": "49.99",
    "isOnSale": "1",
    "savings": "50.010002",
    "metacriticScore": "85",
    "steamRatingText": "Very Positive",
    "steamRatingPercent": "92",
    "steamRatingCount": "5100",
    "steamAppID": "1072420",
    "releaseDate": 1575936000,
    "lastChange": 1688061873,
    "dealRating": "5.6",
    "thumb": "https://cdn.cloudflare.steamstatic.com/steam/apps/1072420/capsule_sm_120.jpg?t=1661972322"
}
{
    "internalName": "DRAGONQUESTXISECHOESOFANELUSIVEAGEDEFINITIVEEDITION",
    "title": "DRAGON QUEST XI S: Echoes of an Elusive Age - Definitive Edition",
    "metacriticLink": "/game/pc/dragon-quest-xi-s-echoes-of-an-elusive-age---definitive-edition",
    "dealID": "

([[{'gameKey': 'DRAGONQUESTBUILDERS2',
    'storeID': 1,
    'title': 'DRAGON QUEST BUILDERS 2',
    'gameID': '207284',
    'salePrice': 49.99,
    'isOnSale': 1,
    'metacriticScore': 85,
    'steamRatingText': 'Very Positive',
    'steamRatingPercent': 92,
    'steamRatingCount': 5100,
    'releaseDate': '1575936000',
    'lastChange': '1688061873',
    'thumb': 'https://cdn.cloudflare.steamstatic.com/steam/apps/1072420/capsule_sm_120.jpg?t=1661972322'},
   {'gameKey': 'DRAGONQUESTXISECHOESOFANELUSIVEAGEDEFINITIVEEDITION',
    'storeID': 1,
    'title': 'DRAGON QUEST XI S: Echoes of an Elusive Age - Definitive Edition',
    'gameID': '218569',
    'salePrice': 39.99,
    'isOnSale': 1,
    'metacriticScore': 0,
    'steamRatingText': 'Very Positive',
    'steamRatingPercent': 93,
    'steamRatingCount': 6036,
    'releaseDate': '1607040000',
    'lastChange': '1688061711',
    'thumb': 'https://cdn.cloudflare.steamstatic.com/steam/apps/1295510/capsule_sm_120.jpg?t=1683878206'},
   

In [38]:
gsa = GameSharkAPI()
#gsa.requestTest()
#gsa.goToDeal("")
gsa.getStoreIds()

#gsa.getMaxDealPages()

def imitateUserInput(gsa):
    userInput = input("Enter game to search: ")
    gsa.searchGame(userInput)
    
#imitateUserInput(gsa)

{'storeID': '1', 'storeName': 'Steam', 'isActive': 1, 'banner': 'https://www.cheapshark.com/img/stores/banners/0.png', 'logo': 'https://www.cheapshark.com/img/stores/logos/0.png', 'icon': 'https://www.cheapshark.com/img/stores/icons/0.png'}
{'storeID': '2', 'storeName': 'GamersGate', 'isActive': 1, 'banner': 'https://www.cheapshark.com/img/stores/banners/1.png', 'logo': 'https://www.cheapshark.com/img/stores/logos/1.png', 'icon': 'https://www.cheapshark.com/img/stores/icons/1.png'}
{'storeID': '3', 'storeName': 'GreenManGaming', 'isActive': 1, 'banner': 'https://www.cheapshark.com/img/stores/banners/2.png', 'logo': 'https://www.cheapshark.com/img/stores/logos/2.png', 'icon': 'https://www.cheapshark.com/img/stores/icons/2.png'}
{'storeID': '4', 'storeName': 'Amazon', 'isActive': 0, 'banner': 'https://www.cheapshark.com/img/stores/banners/3.png', 'logo': 'https://www.cheapshark.com/img/stores/logos/3.png', 'icon': 'https://www.cheapshark.com/img/stores/icons/3.png'}
{'storeID': '5', 'sto

In [26]:
cursor.execute("""
SHOW TABLES;
""")
cursor.fetchall()

[('TestTable',)]